. 
# Performance Optimization

# Parallelism

## Multithreading & Multiprocessing

![](multiprocessing.png)

Python is a hard language to make multithreaded, mostly because of the **global interpreter lock**: Python is an interpreted language, and if the interpreter runs in only one thread, all the nice threads you're producing also run in only one thread.  
While multi*processing* finds a way around that, even multithreading can still be of use. While multithreading can, in Python, not be **parallel**, it can still be **concurrent**. 
* *parallel* processes run truly at the same time - meaning that they must run sumultaneously on different CPU-cores
* *concurrent* processes appear to be parallel to most of the system, even though the CPU may handle them one after another (either parallel or interlocked)

While CPU-intense processes are only truly sped up when they are parallel (something where Python's multithreading doesn't help), tasks that have a bottleneck in network or disk access are helped greatly from cuncurrent execution already.

## Multithreading

To a process, there's a lot of information for the computer:
* State of the process (ready, running, inactive)
* Program-counter (next commans)
* CPU registers (cashed)
* Scheduling information (priority, position)
* Storage information
* I/O-Status
* ...

When switching processes, all those information needs to be saved, such that the CPU can load another process and freeze this one. This produces a lot of overhead. 

**Threads** are lightweight processes, using *shared resources*:
* Shared storage space
* Shared program code
* Shared (virtual) files.

Modern operating systems use threads to let programs switch control, without all the overhead of having to save and load all the information.   
**Advantages**:
* Much faster creation and task switching
* Efficient communication between threads (unlike processes)
* Operating system doesn't schedule them, processes can implement their own scheduling

**Disadvantages**:
* Operating system doesn't schedule them, harder to synchronize than processes
* Processes are better isolated
* Crashing Thread = Crashing Program  
* Python can't use them for parallel processing

In [1]:
import numpy as np
import time

a = np.random.rand(1000000)

def sorter():
    np.sort(a)

start = time.perf_counter()

for i in range(100):
    sorter()

print(time.perf_counter() - start, "seconds")

7.578306792995136 seconds


In [2]:
import threading

a = np.random.rand(1000000)

def sorter():
    np.sort(a)

start = time.perf_counter()

threads = []
for i in range(100):
    t = threading.Thread(target=sorter)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print(time.perf_counter() - start, "seconds")

1.6924289310045424 seconds


Recipe for creating threads:
* Create a **child-thread** by calling the `threading.Thread`-constructor, specifying a **target function**
* Save the reference to that thread somewhere, as you need it later(!!)
* Start the child-thread. After you started it, your **main-thread** can continue doing other things that will done (seemingly) in parallel to the child.
* Before your main-thread ends, `join` all threads, such that they main-thread waits until they are all done, too

In [3]:
import json
import os
from pathlib import Path
from urllib.request import urlopen, Request
from time import time
import pprint
pp = pprint.PrettyPrinter(indent=2)
ALLOWED_FILETYPES = ['.png', '.jpg', '.gif']
flatten = lambda l: [item for sublist in l for item in sublist]

def get_links(client_id):
   def get_img(json):
     if os.path.splitext(json['link']) in ALLOWED_FILETYPES:
        return [json['link']]
     elif 'images' in json:
        return [i['link'] for i in json['images']]
     else:
        return []
    
   headers = {'Authorization': 'Client-ID {}'.format(client_id)}
   all_ims = []
   for page in ['1', '2']:
       req = Request('https://api.imgur.com/3/gallery/hot/viral/month', headers=headers, method='GET')
       with urlopen(req) as resp:
           data = json.loads("\n".join([i.decode('utf-8') for i in resp.readlines()]))
       #pp.pprint(data)
       all_ims = all_ims+[get_img(i) for i in data['data']]
   return flatten(all_ims)


def download_link(directory, link):
   download_path = directory / os.path.basename(link)
   with urlopen(link) as image, download_path.open('wb') as f:
       for line in image.readlines():
            f.write(line)

        
def setup_download_dir():
   download_dir = Path('images')
   if not download_dir.exists():
       download_dir.mkdir()
   return download_dir

In [4]:
from id import client_id

ts = time()
download_dir = setup_download_dir()
links = [l for l in get_links(client_id) if os.path.splitext(l)[1] in ALLOWED_FILETYPES]
for link in links:
   download_link(download_dir, link)
print('Took {}s'.format(time() - ts))

Took 36.493831396102905s


In [5]:
from queue import Queue
from threading import Thread
from id import client_id

class DownloadWorker(Thread):
   def __init__(self, queue):
       Thread.__init__(self)
       self.queue = queue

   def run(self):
       while True:
           # Get the work from the queue and expand the tuple
           directory, link = self.queue.get()
           download_link(directory, link)
           self.queue.task_done()

ts = time()
download_dir = setup_download_dir()
links = [l for l in get_links(client_id) if os.path.splitext(l)[1] in ALLOWED_FILETYPES]

queue = Queue()                    # Create a queue to communicate with the worker threads
for x in range(8):                 # Create 8 worker threads
   worker = DownloadWorker(queue) 
   worker.daemon = True            # A daemon lets the main thread exit even though the workers are blocking
   worker.start()                 

for link in links:
   queue.put((download_dir, link)) # Put the tasks into the queue as a tuple
queue.join()                       # Causes the main thread to wait for the queue to finish processing all the tasks
print('Took {}s'.format(time() - ts))

Took 22.691282033920288


Now, there is a class ``DownloadWorker``, which *is* a Python Thread. On every iteration of it's run, it calls ``self.queue.get()``, which fetches a URL from a thread-safe queue. Once the worker recieves such an item, it calls the ``download_link`` method that we used before. Then the worker must signal the queue that the task is done -- if not, ``queue.join()`` would block the main thread forever.  

Note that while this method is concurrent, it is **not parallel** due to Python's GIL! - it is faster because the IO is the bottleneck in this task! The processor is mostly waiting, and can pick up working on a thread as soon as the network is done.

## Multiprocessing 
If code is performing a CPU-heavy task, the execution time will probably be **slower**!  
For such tasks, we need the ``multiprocessing`` module!

To use multiple processing, what we generally do is to create a multiprocessing ``Pool``, which provides a method ``map``. This method is passed a list of URLs to the pool, which spawns individual processes - processes that can execute our download of the images *truly parallel*.  
As mentioned above, the disadvantage of this method is that the entire memory of the script must be copied to every created subprocess, including all its overhead.

In [6]:
from functools import partial
from multiprocessing.pool import Pool
from id import client_id

ts = time()
download_dir = setup_download_dir()
links = [l for l in get_links(client_id) if os.path.splitext(l)[1] in ALLOWED_FILETYPES]
download = partial(download_link, download_dir)

with Pool(8) as p:
   p.map(download, links)
print('Took {}s'.format(time() - ts))

Took 20.5360369682312s


https://docs.python.org/3/library/multiprocessing.html#using-a-pool-of-workers   
https://docs.python.org/3/library/multiprocessing.html#module-multiprocessing.pool   
https://stackoverflow.com/questions/26520781/multiprocessing-pool-whats-the-difference-between-map-async-and-imap

### map vs imap vs map_async

`map` consumes your iterable by converting the iterable to a list (assuming it isn't a list already), breaking it into chunks, and sending those chunks to the worker processes in the `Pool`.  
Breaking the iterable into chunks performs better than passing each item in the iterable between processes one item at a time - particularly if the iterable is large. However, turning the iterable into a list in order to chunk it can have a very high memory cost, since the entire list will need to be kept in memory. 

In [88]:
import multiprocessing
import time

def func(x):
    time.sleep(x)
    return x + 2

if __name__ == "__main__":    
    p = multiprocessing.Pool()
    start = time.time()
    for x in p.map(func, [1,5,3]):
        print("{} (Time elapsed: {}s)".format(x, int(time.time() - start)))
        
# I am expecting 3, 7, 5 appearing simultaenously

3 (Time elapsed: 5s)
7 (Time elapsed: 5s)
5 (Time elapsed: 5s)


`imap` doesn't turn the iterable you give it into a list, nor does break it into chunks (by default). It will iterate over the iterable one element at a time, and send them each to a worker process.  
This means you don't take the memory hit of converting the whole iterable to a list, but it also means the performance is slower for large iterables, because of the lack of chunking. This can be mitigated by passing a `chunksize` argument larger than default of 1, however.

The other major difference between `imap/imap_unordered` and `map/map_async`, is that with imap/imap_unordered, you can start receiving results from workers as soon as they're ready, rather than having to wait for all of them to be finished.   
With `map_async`, an `AsyncResult` is returned right away, but you can't actually retrieve results from that object until all of them have been processed, at which points it returns the same list that `map` does (map is actually implemented internally as `map_async(...).get()`). There's no way to get partial results; you either have the entire result, or nothing.

In [89]:
import multiprocessing
import time

def func(x):
    time.sleep(x)
    return x + 2

if __name__ == "__main__":    
    p = multiprocessing.Pool()
    start = time.time()
    for x in p.imap(func, [1,5,3]):
        print("{} (Time elapsed: {}s)".format(x, int(time.time() - start)))
        
# I am expecting 3, 7, 5 appearing once they're ready, but in the forced order

3 (Time elapsed: 1s)
7 (Time elapsed: 5s)
5 (Time elapsed: 5s)


`imap` and `imap_unordered` both return iterables right away. With imap, the results will be `yielded` from the iterable as soon as they're ready, while still preserving the ordering of the input iterable. With `imap_unordered`, results will be `yielded` as soon as they're ready, regardless of the order of the input iterable.

In [90]:
import multiprocessing
import time

def func(x):
    time.sleep(x)
    return x + 2

if __name__ == "__main__":    
    p = multiprocessing.Pool()
    start = time.time()
    for x in p.imap_unordered(func, [1,5,3]):
        print("{} (Time elapsed: {}s)".format(x, int(time.time() - start)))
        
# I am expecting 3, 5, 7 appearing once they're ready

3 (Time elapsed: 1s)
5 (Time elapsed: 3s)
7 (Time elapsed: 5s)


Primary reasons to use `imap/imap_unordered` over `map_async` are:
* Your iterable is large enough that converting it to a list would cause you to run out of/use too much memory.
* You want to be able to start processing the results before all of them are completed.

https://stackoverflow.com/questions/35908987/multiprocessing-map-vs-map-async   
https://stackoverflow.com/questions/11338044/python-multiprocessing-whats-the-difference-between-map-and-imap   

# Asynchronous programming: Async&Await

Next to multiprocessing and multithreading for parallel processing, Python also provides the possibility for **asynchronous programming**, a different paradigm for parallel programming, mostly known from Javascript: [https://www.youtube.com/watch?v=3CmKIUmLmJo](https://www.youtube.com/watch?v=3CmKIUmLmJo)  

For an example of that, it is referred to the full version of the source of this code and explanations at [https://www.toptal.com/python/beginners-guide-to-concurrency-and-parallelism-in-python](https://www.toptal.com/python/beginners-guide-to-concurrency-and-parallelism-in-python).

# Numba
Numba is an open source JIT compiler that translates a subset of Python and NumPy code into fast machine code. Numba-compiled numerical algorithms in Python can approach the speeds of C. Instead of having to re-write or even re-compile your code, you just have to add one decorator to the function.

In [7]:
np.arange(100).reshape(10, 10)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
       [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
       [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])

In [8]:
import numpy as np

def go_slow(a):
    trace = 0
    for i in range(a.shape[0]):   
        trace += np.tanh(a[i, i]) 
    return a + trace 

print(go_slow(np.arange(100).reshape(10, 10)))

[[  9.  10.  11.  12.  13.  14.  15.  16.  17.  18.]
 [ 19.  20.  21.  22.  23.  24.  25.  26.  27.  28.]
 [ 29.  30.  31.  32.  33.  34.  35.  36.  37.  38.]
 [ 39.  40.  41.  42.  43.  44.  45.  46.  47.  48.]
 [ 49.  50.  51.  52.  53.  54.  55.  56.  57.  58.]
 [ 59.  60.  61.  62.  63.  64.  65.  66.  67.  68.]
 [ 69.  70.  71.  72.  73.  74.  75.  76.  77.  78.]
 [ 79.  80.  81.  82.  83.  84.  85.  86.  87.  88.]
 [ 89.  90.  91.  92.  93.  94.  95.  96.  97.  98.]
 [ 99. 100. 101. 102. 103. 104. 105. 106. 107. 108.]]


In [21]:
from numba import jit

@jit(nopython=True) 
def go_fast(a):
    trace = 0
    for i in range(a.shape[0]):   
        trace += np.tanh(a[i, i]) 
    return a + trace    

go_fast(np.arange(100).reshape(10, 10)); #we need to compile the function

In [10]:
x = np.arange(10_000).reshape(100, 100)

In [18]:
%%time
go_slow(x)

CPU times: user 942 µs, sys: 0 ns, total: 942 µs
Wall time: 811 µs


array([[   99.,   100.,   101., ...,   196.,   197.,   198.],
       [  199.,   200.,   201., ...,   296.,   297.,   298.],
       [  299.,   300.,   301., ...,   396.,   397.,   398.],
       ...,
       [ 9799.,  9800.,  9801., ...,  9896.,  9897.,  9898.],
       [ 9899.,  9900.,  9901., ...,  9996.,  9997.,  9998.],
       [ 9999., 10000., 10001., ..., 10096., 10097., 10098.]])

In [20]:
%%time 
go_fast(x)

CPU times: user 71 µs, sys: 4 µs, total: 75 µs
Wall time: 77.7 µs


array([[   99.,   100.,   101., ...,   196.,   197.,   198.],
       [  199.,   200.,   201., ...,   296.,   297.,   298.],
       [  299.,   300.,   301., ...,   396.,   397.,   398.],
       ...,
       [ 9799.,  9800.,  9801., ...,  9896.,  9897.,  9898.],
       [ 9899.,  9900.,  9901., ...,  9996.,  9997.,  9998.],
       [ 9999., 10000., 10001., ..., 10096., 10097., 10098.]])

# Cython
One way to make your program run faster: compile to a faster language.
[https://cython.org/](https://cython.org/)

### About Cython

Cython is an optimising static compiler for both the Python programming language and the extended Cython programming language (based on Pyrex). It makes writing C extensions for Python as easy as Python itself.

Cython gives you the combined power of Python and C to let you

* write Python code that calls back and forth from and to C or C++ code natively at any point.
* easily tune readable Python code into plain C performance by adding static type declarations, also in Python syntax.
* use combined source code level debugging to find bugs in your Python, Cython and C code.
* interact efficiently with large data sets, e.g. using multi-dimensional NumPy arrays.
* quickly build your applications within the large, mature and widely used CPython ecosystem.
* integrate natively with existing code and data from legacy, low-level or high-performance libraries and applications.

The Cython language is a superset of the Python language that additionally supports calling C functions and declaring C types on variables and class attributes. This allows the compiler to generate very efficient C code from Cython code. The C code is generated once and then compiles with all major C/C++ compilers in CPython 2.6, 2.7 (2.4+ with Cython 0.20.x) as well as 3.3 and all later versions. We regularly run integration tests against all supported CPython versions and their latest in-development branches to make sure that the generated code stays widely compatible and well adapted to each version. PyPy support is work in progress (on both sides) and is considered mostly usable since Cython 0.17. The latest PyPy version is always recommended here.

All of this makes Cython the ideal language for wrapping external C libraries, embedding CPython into existing applications, and for fast C modules that speed up the execution of Python code. 

## Intro 

[https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html](https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html) 

The fundamental nature of Cython can be summed up as follows: Cython is Python with C data types.

Cython is Python: Almost any piece of Python code is also valid Cython code. (There are a few Limitations, but this approximation will serve for now.) The Cython compiler will convert it into C code which makes equivalent calls to the Python/C API.

But Cython is much more than that, because parameters and variables can be declared to have C data types. Code which manipulates Python values and C values can be freely intermixed, with conversions occurring automatically wherever possible. Reference count maintenance and error checking of Python operations is also automatic, and the full power of Python’s exception handling facilities, including the try-except and try-finally statements, is available to you – even in the midst of manipulating C data.

For an introduction on how to use Cython, I recommend [reading the docs.](https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html)

## Example

In [33]:
def fib_python(n):
    """Print the Fibonacci series up to n."""
    a, b = 0, 1
    while b < n:
        print(b, end=' ')
        a, b = b, a + b

    print()

In [79]:
%%time 
fib_python(1e+10)

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155 165580141 267914296 433494437 701408733 1134903170 1836311903 2971215073 4807526976 7778742049 
CPU times: user 15 ms, sys: 4.2 ms, total: 19.2 ms
Wall time: 14.5 ms


In [80]:
import sys
sys.path.append('./cython')
from fibonacci import fib

In [81]:
%%time 
fib.fib(1e+10)

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155 165580141 267914296 433494437 701408733 1134903170 1836311903 2971215073 4807526976 7778742049 
CPU times: user 2.89 ms, sys: 162 µs, total: 3.05 ms
Wall time: 2.05 ms


# Distributed Computing

Another way to make it faster: make a computation graph & distribute. As we don't have time for this, I recommend looking into [Dask](https://dask.org/) if you're interested.